In [9]:
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
import ssl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings("ignore")

In [10]:
BASE_DIR = Path.cwd().parent
nltk.download('vader_lexicon')
ssl._create_default_https_context = ssl._create_unverified_context

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/tatiana.ilyasova/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
df_hosts_reviews_en = pd.read_parquet(BASE_DIR / 'staging_data' / 'hosts_reviews_en_cleaned.parquet')

In [12]:
df_hosts_reviews_en

,index,date,listing_id_encod,listing_id,reviewer_id,reviewer_name,comments,host_id,host_name,host_description,...,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,comments_language,description_language
0,0,2010-09-06,0.0,3.298400e+04,200247.0,Keenan,great little apartment perfect spot town nice ...,53396.0,Chris,city hall stockholm beautiful bright 43 sqm 1b...,...,0.0,4.23,4.43,3.81,4.29,4.52,4.81,4.20,en,en
1,1,2010-09-12,0.0,3.298400e+04,126543.0,Olaf,nice flat great area host sorted things make s...,53396.0,Chris,city hall stockholm beautiful bright 43 sqm 1b...,...,0.0,4.23,4.43,3.81,4.29,4.52,4.81,4.20,en,en
2,2,2011-05-21,0.0,3.298400e+04,294223.0,Max,great location host responsive email country t...,53396.0,Chris,city hall stockholm beautiful bright 43 sqm 1b...,...,0.0,4.23,4.43,3.81,4.29,4.52,4.81,4.20,en,en
3,3,2011-05-28,0.0,3.298400e+04,577190.0,Eli,host place super great close everything quite ...,53396.0,Chris,city hall stockholm beautiful bright 43 sqm 1b...,...,0.0,4.23,4.43,3.81,4.29,4.52,4.81,4.20,en,en
4,4,2011-06-11,0.0,3.298400e+04,521818.0,Jonathan,nice apartment great location great price host...,53396.0,Chris,city hall stockholm beautiful bright 43 sqm 1b...,...,0.0,4.23,4.43,3.81,4.29,4.52,4.81,4.20,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69626,92991,2023-03-12,2768.0,8.259831e+17,238711214.0,Helena Sofie,host really great host place clean area stay g...,6839482.0,Göran,room2stay lev det enkla livet detta fridfulla ...,...,4.0,5.00,4.75,5.00,4.75,4.75,4.75,4.75,en,sv
69627,92992,2023-03-19,2768.0,8.259831e+17,490919229.0,Dahyoon,lucky meet host always said us make home reall...,6839482.0,Göran,room2stay lev det enkla livet detta fridfulla ...,...,4.0,5.00,4.75,5.00,4.75,4.75,4.75,4.75,en,sv
69628,92994,2023-02-22,2769.0,8.268185e+17,52396317.0,Ivar,beautiful apartment central everything functio...,51002540.0,Fredrik,spacious luxurious 2br apt östermalmstorg enjo...,...,4.0,3.75,3.50,4.75,3.75,4.25,4.50,3.50,en,en
69629,92996,2023-03-10,2769.0,8.268185e+17,112662633.0,Leonora,apartment absolutely beautiful clean nice host...,51002540.0,Fredrik,spacious luxurious 2br apt östermalmstorg enjo...,...,4.0,3.75,3.50,4.75,3.75,4.25,4.50,3.50,en,en


In [13]:
df_hosts_reviews_en.groupby('review_scores_rating')['comments'].count().sort_values(ascending=False)

review_scores_rating
5.00    4539
4.92    3520
4.84    2407
4.83    2371
4.86    2195
        ... 
3.40       4
2.50       4
3.75       3
2.00       2
1.50       1
Name: comments, Length: 114, dtype: int64

In [14]:
df_hosts_reviews_en.query('review_scores_rating < 3')

,index,date,listing_id_encod,listing_id,reviewer_id,reviewer_name,comments,host_id,host_name,host_description,...,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,comments_language,description_language
24959,31667,2017-08-05,638.0,1.836942e+07,13333455.0,Lara,host nice person available time even asked sev...,17283764.0,Madelaine,familjevänlig lägenhet mkt centralt på kungsho...,...,0.0,2.0,1.0,1.0,4.00,5.0,5.00,1.0,en,sv
48883,63647,2019-07-15,1285.0,3.593015e+07,86501510.0,David,apartment welllocated metro 5 min walk host ni...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48884,63648,2019-07-24,1285.0,3.593015e+07,193898979.0,Ella,hes nice kinda lacking everything else gues ap...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48885,63649,2019-07-28,1285.0,3.593015e+07,22855458.0,Salma,good location price however room rented one de...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
48886,63650,2022-10-03,1285.0,3.593015e+07,13501270.0,Haizhou,communication wait apartment without anyone sh...,270253234.0,Mehdi,room large apartment rent july en trea med 75 ...,...,1.0,2.5,3.5,2.5,3.25,3.5,3.75,3.0,en,sv
49373,64304,2022-06-26,1324.0,3.675865e+07,341209083.0,"O,B",house really dirty smelled horrible,72819688.0,Sam,big 2 room apartment big 2 room apartment loca...,...,1.0,1.0,1.0,1.0,1.00,1.0,1.00,1.0,en,en
52479,68753,2022-08-25,1362.0,3.758750e+07,476240802.0,Jan,dont book phone number host assigned host answer,283964182.0,Sofia,sovrum med tillgång till hela lägenheten vid b...,...,1.0,1.0,1.0,1.0,1.00,1.0,1.00,1.0,en,sv
56126,73993,2022-08-10,1574.0,4.354639e+07,98082896.0,Daniel,dont stay unless want use place sleep wont use...,67355594.0,Pernilla,lustiga huset hökarängen etagelägenhet med tak...,...,1.0,1.0,1.0,1.0,4.00,2.0,4.00,1.0,en,sv
65099,86349,2022-06-21,2093.0,5.788862e+17,220587752.0,Jonathan,walked supposed smokefree unit reeked smoke sm...,310835509.0,TravelNest,rsta 342 34 bed apartment stockholm discover s...,...,2.0,1.5,2.0,1.5,2.00,2.0,1.50,1.5,en,en
68458,91173,2022-11-01,2497.0,6.924256e+17,452097355.0,Stella,worst experience host since use platform askin...,36019300.0,These,mysig lägenhet med uteplats mitt city välkomme...,...,1.0,2.0,3.0,2.0,1.00,3.0,5.00,2.0,en,sv


In [15]:
analyzer = SentimentIntensityAnalyzer()
def label_sentiment(df, text_column):
    """
    Add a new column to the input dataframe with the sentiment label for each text.

    Parameters:
    df (pandas.DataFrame): The dataframe to process.
    text_column (str): The name of the text column to analyze.

    Returns:
    None
    """
    df['sentiment_label'] = df[text_column].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    df['sentiment_label_class'] = np.where(df['sentiment_label'] > 0, 1, 0)
label_sentiment(df_hosts_reviews_en, 'comments')

In [16]:
df_hosts_reviews_en.to_parquet(BASE_DIR / 'processed_data' / 'hosts_reviews_en_labeled.parquet')